# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import cartopy.crs as ccrs
import geoviews as gv 

# Import API key
from api_keys import geoapify_key

ModuleNotFoundError: No module named 'cartopy'

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data:cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,goure,13.9835,10.2704,65.35,19,33,10.11,NE,2023-02-09 00:07:45
1,1,punta arenas,-53.1500,-70.9167,55.51,50,40,19.57,CL,2023-02-09 00:07:46
2,2,walvis bay,-22.9575,14.5053,65.08,90,5,8.14,NaN,2023-02-09 00:07:46
3,3,yerbogachen,61.2767,108.0108,-6.12,99,100,4.88,RU,2023-02-09 00:07:47
4,4,ushuaia,-54.8000,-68.3000,51.46,93,100,5.75,AR,2023-02-09 00:07:47


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600
)

# Display the map
map_plot_1

ImportError: Geographic projection support requires GeoViews, pyproj and cartopy.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) 
                                    & (city_data_df["Max Temp"] > 70) 
                                    & (city_data_df["Wind Speed"] < 10) 
                                    & (city_data_df["Cloudiness"] == 0),:].dropna()

# Drop any rows with null values

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
70,70,pedernales,18.0384,-71.7440,77.59,67,0,6.13,DO,2023-02-09 00:08:11
147,147,santa ana,33.7456,-117.8678,76.95,37,0,9.22,US,2023-02-09 00:07:44
167,167,mar del plata,-38.0023,-57.5575,72.09,86,0,4.00,AR,2023-02-09 00:08:48
220,220,saint-pierre,-21.3393,55.4781,74.88,64,0,5.75,RE,2023-02-09 00:09:02
313,313,coihueco,-36.6167,-71.8333,74.55,29,0,3.27,CL,2023-02-09 00:09:41


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_weather_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
70,70,pedernales,18.0384,-71.7440,77.59,67,0,6.13,DO,2023-02-09 00:08:11,
147,147,santa ana,33.7456,-117.8678,76.95,37,0,9.22,US,2023-02-09 00:07:44,
167,167,mar del plata,-38.0023,-57.5575,72.09,86,0,4.00,AR,2023-02-09 00:08:48,
220,220,saint-pierre,-21.3393,55.4781,74.88,64,0,5.75,RE,2023-02-09 00:09:02,
313,313,coihueco,-36.6167,-71.8333,74.55,29,0,3.27,CL,2023-02-09 00:09:41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": 10000,
    "type": "hotel",
    "keyword": "hotels",
    "key": geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"]
    longitude = hotel_df["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["location"] = f"{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = hotel_name = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = hotel_name.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{
    "error": "Unauthorized",
    "message": "Invalid apiKey",
    "statusCode": 401
}
pedernales - nearest hotel: No hotel found
{
    "error": "Unauthorized",
    "message": "Invalid apiKey",
    "statusCode": 401
}
santa ana - nearest hotel: No hotel found
{
    "error": "Unauthorized",
    "message": "Invalid apiKey",
    "statusCode": 401
}
mar del plata - nearest hotel: No hotel found
{
    "error": "Unauthorized",
    "message": "Invalid apiKey",
    "statusCode": 401
}
saint-pierre - nearest hotel: No hotel found
{
    "error": "Unauthorized",
    "message": "Invalid apiKey",
    "statusCode": 401
}
coihueco - nearest hotel: No hotel found
{
    "error": "Unauthorized",
    "message": "Invalid apiKey",
    "statusCode": 401
}
san clemente - nearest hotel: No hotel found
{
    "error": "Unauthorized",
    "message": "Invalid apiKey",
    "statusCode": 401
}
russell - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
70,70,pedernales,18.0384,-71.7440,77.59,67,0,6.13,DO,2023-02-09 00:08:11,No hotel found
147,147,santa ana,33.7456,-117.8678,76.95,37,0,9.22,US,2023-02-09 00:07:44,No hotel found
167,167,mar del plata,-38.0023,-57.5575,72.09,86,0,4.00,AR,2023-02-09 00:08:48,No hotel found
220,220,saint-pierre,-21.3393,55.4781,74.88,64,0,5.75,RE,2023-02-09 00:09:02,No hotel found
313,313,coihueco,-36.6167,-71.8333,74.55,29,0,3.27,CL,2023-02-09 00:09:41,No hotel found
369,369,san clemente,33.4270,-117.6120,76.64,41,0,4.00,US,2023-02-09 00:08:05,No hotel found
524,524,russell,32.3502,-85.1999,72.63,64,0,5.75,US,2023-02-09 00:06:47,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot

map_plot_1 = city_data_df.hvplot.points(
    "Lon",
    "Lat",
    hover_cols=["Hotel Name", "country"],
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
)

# Display the map
mat_plot_1.show()

ImportError: Geographic projection support requires GeoViews, pyproj and cartopy.